<a href="https://colab.research.google.com/github/postusername/deepfake-tg/blob/master/Summarization_IT_Cube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Решение задачи суммаризации

## Работа с данными

#### Качаем датасет

In [ ]:
!gdown 1dpq77cRn47M6qtM3Lgv0aHRphHkcxRdz
!unzip summarization.zip

Downloading...
From: https://drive.google.com/uc?id=1dpq77cRn47M6qtM3Lgv0aHRphHkcxRdz
To: /content/summarization.zip
100% 6.42M/6.42M [00:00<00:00, 160MB/s]
Archive:  summarization.zip
  inflating: summarization/data.xlsx  
  inflating: summarization/README.md  


In [ ]:
!cd /content/summarization/
!cat /content/summarization/README.md

Описание датасета:
24039 обращений, разбитые на три столбца:
1. id - ид обращения в датасете
2. description - краткое описание первого письма
3. first_message - письмо, с которым обратился клиент в службу поддержки (т.е. первое письмо из переписки по обращению).

Проблемы датасета:
- поля first_message у обращений бывают пустые. это специфика некоторых обращений: их можно завести через сайт, создав шаблонное обращение - например, запрос ключа лицензии на наш продукт. такое обращение создаётся с пустым текстом, но через синхронизацию с сайтом у нас подхватывается тип обращения и инженер поддержки получает оформленное обращение, хоть и фактически первое письмо пустое. такие обращения чаще всего будут относиться и ко второй проблеме, т.к. с пустыми письмами бывают чаще всего запросы.
- есть поля description типа "Запрос [...]". эти обращения не относятся к суммаризационным и по идее добавлены в датасет __ошибочно__ (т.к. датасет составлял не специалист по данным). в чём суть проблемы: у т

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_excel('/content/summarization/data.xlsx')#, index_col=0)
df['description'] = df['description'].str.lower()

In [ ]:
df

,id,description,first_message
0,0,запрос разрешения на генерацию ключа,"Добрый день. Умер сервер, на котором работала..."
1,1,запрос разрешения на генерацию ключа,NaN
2,2,запрос разрешения на генерацию ключа,Добрый день! Прошу предоставить разрешение...
3,3,запрос обновления системы,Добрый день. Прошу предоставить пакет обновле...
4,4,запрос разрешения на генерацию ключа,Прошу предоставить разрешение на генерацию кл...
...,...,...,...
24035,24035,"""при обращении к серверу приложений возникла о...","Здравствуйте! У клиента возникла ошибка, при ..."
24036,24036,как соотнести пользователей в ad с существующи...,Необходима консультация по решению «Интеграци...
24037,24037,ошибка: у пользователей некорректное время в з...,Directum RX 4.1 Правительства Тюменской облас...
24038,24038,при сканировании результат выходит перевернуты...,NaN


#### Удаление лишних пробелов и пустых данных

In [ ]:
def clear_spaces(x: str):
  new = x.replace('\t', '').replace('  ', ' ').replace('\n', ' ').strip().lower()
  if len(new) < 3:
    return np.nan
  else:
    return new

In [ ]:
df = df.dropna()
df['description'] = [clear_spaces(x) for x in df['description']]
df['first_message'] = [clear_spaces(x) for x in df['first_message']]
df = df.dropna()

In [ ]:
df

,id,description,first_message
0,0,запрос разрешения на генерацию ключа,"добрый день. умер сервер, на котором работала ..."
2,2,запрос разрешения на генерацию ключа,добрый день! прошу предоставить разрешение на ...
3,3,запрос обновления системы,добрый день. прошу предоставить пакет обновлен...
4,4,запрос разрешения на генерацию ключа,прошу предоставить разрешение на генерацию клю...
5,5,запрос разрешения на генерацию ключа,продолжение обращения № 317320. возможно ли по...
...,...,...,...
24034,24034,возможно ли на моменте выгрузки корректировать...,необходима консультация по решению «интеграция...
24035,24035,"""при обращении к серверу приложений возникла о...","здравствуйте! у клиента возникла ошибка, при р..."
24036,24036,как соотнести пользователей в ad с существующи...,необходима консультация по решению «интеграция...
24037,24037,ошибка: у пользователей некорректное время в з...,directum rx 4.1 правительства тюменской област...


#### Очистка от повторов

Чтобы убрать повторяющиеся "*запросы*", обучим классификатор и будем перед суммаризацией определять, типовое обращение или нет.
Для начала, вытащим все часто повторяющиеся темы.

In [ ]:
desc = dict(df['description'].value_counts())

trash = []
for x in df['description'].unique():
  # std / mean по всей выборке со словом Запрос — 10.43
  if x.lower().startswith("запрос ") and desc[x] > 10:
    trash.append(x)

In [ ]:
trash_classes = dict()
for i in range(len(trash)):
  trash_classes.update({trash[i]: i+1})

In [ ]:
typical = [df[df['description'] == template]['id'] for template in trash]
typical = pd.concat(typical).values # массив id обращений с повторяющимися темами

In [ ]:
template_data = []
sum_data = []
for row in df.values:
  if row[0] in typical:
    template_data.append([row[2], trash_classes[row[1].lower()]])
  else:
    template_data.append([row[2], 0])
    sum_data.append(row)
template_data = pd.DataFrame(template_data, columns=['text', 'template'])
sum_data = pd.DataFrame(sum_data, columns=['id', 'desc', 'text'])

In [ ]:
template_data

,text,template
0,"добрый день. умер сервер, на котором работала ...",1
1,добрый день! прошу предоставить разрешение на...,1
2,добрый день. прошу предоставить пакет обновлен...,2
3,прошу предоставить разрешение на генерацию клю...,1
4,продолжение обращения № 317320. возможно ли по...,1
...,...,...
23022,необходима консультация по решению «интеграция...,0
23023,"здравствуйте! у клиента возникла ошибка, при р...",0
23024,необходима консультация по решению «интеграция...,0
23025,directum rx 4.1 правительства тюменской област...,0


Теперь подготовка этих микроданных и обучение классификатора.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
sw = stopwords.words()

def tokenize_text(txt, min_token_size=4):
    all_tokens = word_tokenize(txt.lower())
    tokens = []
    for token in all_tokens:
      if (len(token) >= min_token_size) and (not token in sw) and (token.isalpha()):
        tokens.append(token)
    return tokens

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(template_data, test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, f1_score, matthews_corrcoef, make_scorer

# в скольких видах документов должен встречаться термин, чтобы мы его не учитывали
MAX_DF = 0.95
# сколько раз слово должно встретиться в документе, чтобы мы его учитывали
MIN_COUNT = 4

sklearn_pipeline = Pipeline((('vect', TfidfVectorizer(tokenizer=tokenize_text,
                                                      max_df=MAX_DF,
                                                       min_df=MIN_COUNT)),
                            #('cls', MLPClassifier((150,), activation='logistic', solver='lbfgs', max_iter=3000, verbose=True))), verbose=True)
                            ('cls', RandomForestClassifier(max_features=None))))
sklearn_pipeline.fit(train_data['text'], train_data['template'])

Pipeline(steps=[('vect',
                 TfidfVectorizer(max_df=0.95, min_df=4,
                                 tokenizer=<function tokenize_text at 0x7f573ac6eef0>)),
                ('cls', RandomForestClassifier(max_features=None))])

In [ ]:
from sklearn.metrics import log_loss, f1_score, matthews_corrcoef

p = sklearn_pipeline.predict(test_data['text'])
pb = sklearn_pipeline.predict_proba(test_data['text'])
print("F1: ", round(f1_score(test_data['template'], p, labels=[x for x in range(10)], average='weighted'), 3))
print("Log-loss: ", round(log_loss(test_data['template'], pb, labels=[x for x in range(10)]), 3))
print("MCC: ", round(matthews_corrcoef(test_data['template'], p), 3))#, labels=[x for x in range(10)]), 3))

F1:  0.818
Log-loss:  1.112
MCC:  0.657


## Модели суммаризации

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)
train, validate = train_test_split(train, test_size=0.25, random_state=42)

#### ***Статистическая*** экстрактивная модель
Самый простой вариант реализации: подсчитать частоту встречаемости слов, сделать поправку на общую употребляемость в тексте и посчитать сумму частот в каждом предложении. Предложение с максимальной суммой содержит слова, которые относительно часто встречаются в тексте.

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

def _create_dictionary_table(text_string) -> dict:
    #removing stop words
    stop_words = set(stopwords.words("english"))
    
    words = word_tokenize(text_string)
    
    #reducing words to their root form
    stem = PorterStemmer()
    
    #creating dictionary for the word frequency table
    frequency_table = dict()
    for wd in words:
        wd = stem.stem(wd)
        if wd in stop_words:
            continue
        if wd in frequency_table:
            frequency_table[wd] += 1
        else:
            frequency_table[wd] = 1

    return frequency_table


def _calculate_sentence_scores(sentences, frequency_table) -> dict:   
    #algorithm for scoring a sentence by its words
    sentence_weight = dict()

    for sentence in sentences:
        sentence_wordcount = (len(word_tokenize(sentence)))
        sentence_wordcount_without_stop_words = 0
        for word_weight in frequency_table:
            if word_weight in sentence.lower():
                sentence_wordcount_without_stop_words += 1
                if sentence[:7] in sentence_weight:
                    sentence_weight[sentence[:7]] += frequency_table[word_weight]
                else:
                    sentence_weight[sentence[:7]] = frequency_table[word_weight]

        sentence_weight[sentence[:7]] = sentence_weight[sentence[:7]] / sentence_wordcount_without_stop_words

    return sentence_weight


def _calculate_average_score(sentence_weight: dict) -> int:
    #calculating the average score for the sentences
    sum_values = 0
    for entry in sentence_weight:
        sum_values += sentence_weight[entry]

    #getting sentence average value from source text
    average_score = (sum_values / len(sentence_weight))

    return average_score


def _get_article_summary(sentences, sentence_weight, threshold):
    sentence_counter = 0
    article_summary = ''

    for sentence in sentences:
        if sentence[:7] in sentence_weight and sentence_weight[sentence[:7]] >= (threshold):
            article_summary += " " + sentence
            sentence_counter += 1

    return article_summary

def statistical_summary(article):
    #creating a dictionary for the word frequency table
    frequency_table = _create_dictionary_table(article)
    #tokenizing the sentences
    sentences = sent_tokenize(article)
    #algorithm for scoring a sentence by its words
    sentence_scores = _calculate_sentence_scores(sentences, frequency_table)
    #getting the threshold
    threshold = _calculate_average_score(sentence_scores)
    #producing the summary
    article_summary = _get_article_summary(sentences, sentence_scores, 1.5 * threshold)

    return article_summary

#### BERT

In [ ]:
!pip install bert-extractive-summarizer > /dev/null
!pip install sacremoses > /dev/null

In [ ]:
from summarizer import Summarizer
bmodel = Summarizer()

def BERT_summary(message):
  return bmodel(message)

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

#### SBERT

In [ ]:
!pip install -U sentence-transformers > /dev/null

In [ ]:
from summarizer.sbert import SBertSummarizer
model_s = SBertSummarizer('all-mpnet-base-v2')

def SBERT_summary(message):
  return model_s(message)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

#### TextRank
Использование алгоритма [TextRank](https://arxiv.org/pdf/1602.03606.pdf) из библиотеки Gensim

In [ ]:
!pip install gensim > /dev/null

In [ ]:
from gensim.summarization.summarizer import summarize

def TR_summary(message):
  return summarize(message)

## Финальные результаты

In [ ]:
!pip install rouge-score > /dev/null
!pip install prettytable > /dev/null
!pip install jiwer > /dev/null

In [ ]:
from rouge_score.rouge_scorer import RougeScorer
from jiwer import wer
scorer = RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)


models = { "Statistical": statistical_summary, 
           "BERT": BERT_summary, 
           "Sentence-BERT": SBERT_summary,
           "TextRank": TR_summary }

In [ ]:
from prettytable import PrettyTable
mytable = PrettyTable()
mytable.field_names = ["Model", "Rouge-1", "Rouge-2", "Rouge-L", "WER accuracy"]

for name, model in list(models.items()):
  metrics = [0, 0, 0, 0]
  for row in test.values:
    try:
      output = model(row[2].replace('. ', '.\n').replace('.', '\n'))
    except ValueError:
      continue
    except ZeroDivisionError:
      continue
    loss = scorer.score(row[1], output)
    loss = list(loss.values())
    for i in range(len(loss)):
      metrics[i] += loss[i].fmeasure
    metrics[-1] += (wer(row[1], output)/4)
  for i in range(len(metrics)):
      metrics[i] /= test.shape[0]
      metrics[i] = round(metrics[i], 3)

  mytable.add_rows([[name, metrics[0], metrics[1], metrics[2], metrics[3]]])

print("Mean f1-values was used for ROUGE.")
table = mytable.get_string(sortby="Rouge-L")
print(table)

Mean f1-values was used for ROUGE.
+---------------+---------+---------+---------+--------------+
|     Model     | Rouge-1 | Rouge-2 | Rouge-L | WER accuracy |
+---------------+---------+---------+---------+--------------+
|  Statistical  |   0.0   |   0.0   |   0.0   |    0.255     |
|    TextRank   |  0.058  |  0.025  |  0.057  |    0.353     |
| Sentence-BERT |   0.19  |  0.123  |  0.188  |    0.746     |
|      BERT     |   0.19  |  0.123  |  0.189  |    0.745     |
+---------------+---------+---------+---------+--------------+


Mean f1-values was used for ROUGE.
+---------------+---------+---------+---------+--------------+
|     Model     | Rouge-1 | Rouge-2 | Rouge-L | WER accuracy |
+---------------+---------+---------+---------+--------------+
|  Statistical  |   0.0   |   0.0   |   0.0   |    0.548     |
|    TextRank   |  0.084  |   0.03  |  0.081  |    0.753     |
|      BERT     |  0.283  |  0.178  |  0.281  |     0.96     |
| Sentence-BERT |  0.283  |  0.178  |  0.281  |     0.96     |
+---------------+---------+---------+---------+--------------+

In [ ]:
row

array([16757, 'запрос разрешения на генерацию ключа', '.'], dtype=object)